In [6]:
import pandas as pd
import numpy as np

In [7]:
nfft = 512
hop_size = 256

In [8]:
bicoh_train_feat_path = '../features/bicoherences/dataframes/train_bicoh_stats_nfft_{}_hop_size_{}.pkl'.format(
    nfft, hop_size)
lpc_train_feat_path =  '../features/lpc/dataframe/train.pkl'
unet_train_feat_path = '../features/unet/train_nfft_{}_hop_size_{}.pkl'.format(nfft, hop_size)


bicoh_dev_feat_path = '../features/bicoherences/dataframes/dev_bicoh_stats_nfft_{}_hop_size_{}.pkl'.format(
    nfft, hop_size)
lpc_dev_feat_path =  '../features/lpc/dataframe/dev.pkl'
unet_dev_feat_path = '../features/unet/dev_nfft_{}_hop_size_{}.pkl'.format(nfft, hop_size)


bicoh_eval_feat_path = '../features/bicoherences/dataframes/eval_bicoh_stats_nfft_{}_hop_size_{}.pkl'.format(
    nfft, hop_size)
lpc_eval_feat_path =  '../features/lpc/dataframe/eval.pkl'
unet_eval_feat_path = '../features/unet/eval_nfft_{}_hop_size_{}.pkl'.format(nfft, hop_size)


In [9]:
selected_features = ['lpc', 'bicoh', 'unet']

number_lpc_order = 49
stop_lpc_order = 50

lpc_linspace = np.linspace(start=stop_lpc_order-number_lpc_order, stop=stop_lpc_order, dtype=int)

lpc_selected_orders = []
for l in lpc_linspace:
    lpc_selected_orders.append(str(l))
    
lpc_selected_orders = tuple(lpc_selected_orders)


In [34]:
train_features = pd.DataFrame()
dev_features = pd.DataFrame()
eval_features = pd.DataFrame()

for feat in selected_features:
    if feat == 'lpc':
        lpc_feat_train = pd.read_pickle(lpc_train_feat_path)
        lpc_feat_train.set_index('audio_filename', inplace=True)

        lpc_feat_dev = pd.read_pickle(lpc_dev_feat_path)
        lpc_feat_dev.set_index('audio_filename', inplace=True)

        lpc_feat_eval = pd.read_pickle(lpc_eval_feat_path)
        lpc_feat_eval.set_index('audio_filename', inplace=True)

        drop_list = [a for a in lpc_feat_train.columns if a.startswith(('lpc', 'ltp')) 
                 and not a.endswith(lpc_selected_orders)]
        
        lpc_feat_train = lpc_feat_train.drop(drop_list, axis=1)
        lpc_feat_dev = lpc_feat_dev.drop(drop_list, axis=1)
        lpc_feat_eval = lpc_feat_eval.drop(drop_list, axis=1)

        if train_features.empty:
            train_features = lpc_feat_train.copy()
            
            dev_features = lpc_feat_dev.copy()
            
            eval_features = lpc_feat_eval.copy()
        else:
            train_features = pd.concat([train_features, lpc_feat_train], axis=1)
            dev_features = pd.concat([dev_features, lpc_feat_dev], axis=1)
            eval_features = pd.concat([eval_features, lpc_feat_eval], axis=1)
        
    elif feat == 'bicoh':
        bicoh_feat_train = pd.read_pickle(bicoh_train_feat_path)
        bicoh_feat_train.set_index('audio_filename', inplace=True)

        bicoh_feat_dev = pd.read_pickle(bicoh_dev_feat_path)
        bicoh_feat_dev.set_index('audio_filename', inplace=True)

        bicoh_feat_eval = pd.read_pickle(bicoh_eval_feat_path)
        bicoh_feat_eval.set_index('audio_filename', inplace=True)

        if train_features.empty:
            train_features = bicoh_feat_train.copy()
            
            dev_features = bicoh_feat_dev.copy()
            
            eval_features = bicoh_feat_eval.copy()
        else:
            train_features = pd.concat([train_features, bicoh_feat_train], axis=1)
            dev_features = pd.concat([dev_features, bicoh_feat_dev], axis=1)
            eval_features = pd.concat([eval_features, bicoh_feat_eval], axis=1)
        
    elif feat== 'unet':
        unet_feat_train = pd.read_pickle(unet_train_feat_path)
        unet_feat_train.set_index('audio_filename', inplace=True)

        unet_feat_dev = pd.read_pickle(unet_dev_feat_path)
        unet_feat_dev.set_index('audio_filename', inplace=True)

        unet_feat_eval = pd.read_pickle(unet_eval_feat_path)
        unet_feat_eval.set_index('audio_filename', inplace=True)


        if train_features.empty:
            train_features = unet_feat_train.copy()
            
            dev_features = unet_feat_dev.copy()
            
            eval_features = unet_feat_eval.copy()
        else:
            train_features = pd.concat([train_features, unet_feat_train], axis=1)
            dev_features = pd.concat([dev_features, unet_feat_dev], axis=1)
            eval_features = pd.concat([eval_features, unet_feat_eval], axis=1)
            
            
train_features = train_features.loc[:,~train_features.columns.duplicated()]
dev_features = dev_features.loc[:,~dev_features.columns.duplicated()]
eval_features = eval_features.loc[:,~eval_features.columns.duplicated()]


train_features.reset_index(inplace=True)
dev_features.reset_index(inplace=True)
eval_features.reset_index(inplace=True)

In [23]:
y_train_open_set = train_features.loc[:, 'system_id']

In [35]:
train_features.drop(['audio_filename', 'end_voice', 'start_voice', 'speaker_id', 'label'], axis=1, inplace=True)

train_features.loc[:,train_features.columns != 'system_id' ].values

array([[334.88987594,  17.98628083,   1.00006594, ...,   0.93589389,
         81.66677856,  16.1964016 ],
       [182.07341034,  25.10167195,   1.00003391, ...,   1.47024941,
        114.99207306,  22.69653511],
       [143.27429727,  18.54383241,   1.07217926, ...,   1.6751529 ,
        119.53869629,  24.00307083],
       ...,
       [ 75.01783315,  16.77978971,   1.03098749, ...,   0.51156944,
         54.409935  ,  10.83079052],
       [238.954941  ,  29.72829882,   1.00355044, ...,   0.64936137,
         64.2545929 ,  12.26148415],
       [264.36180231,  19.63717774,   1.00628034, ...,   0.74547541,
         68.71363831,  13.44063091]])